# Part-of-speech model training
Part-of-speech (POS) tagging is the process of assigning a part-of-speech to each word in a text.
Words are ambiguous — have more than one possible part-of-speech. The goal is to find the correct tag depending on the context.
For example, book can be a verb (book that flight) or a noun (hand me that book).

Why this task is important and why do we care about part-of-speech? Because the semantic of the word (the meaning of the word) could change by changing the part-of-speech of the word from a verb to a noun, as we have seen above.

## Dataset preparation

In [ ]:
from datasets import load_dataset

dataset = load_dataset('conll2003')  # download a dataset

# now you can preview a few examples
dataset['test'].to_pandas().head(5)

In [ ]:
from transformers import AutoTokenizer, GPT2TokenizerFast

# model_checkpoint = "distilbert-base-uncased"  # 66M large model
model_checkpoint = "xlm-roberta-large"  # 558M large model
# model_checkpoint = "gpt2-xl"  # 1.5B large model, possible to fine-tune using adapters, but only with batch size 1
# model_checkpoint = "gpt2-large"  # 774M large model, the same here - only batch size 1 works, maybe because of paddings. Sequence length is longers

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
label_list = dataset["train"].features[f"pos_tags"].feature.names

In [ ]:
def tokenize_and_align_labels(examples):
    """
    After tokenization, a word is split into multiple tokens. This function assigns the same POS tag for every token of the word.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"pos_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenize_and_align_labels(dataset['train'][:5])

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 8
early_stopping_patience = 5
eval_steps = 500
num_epochs = 40
learning_rate = 2e-5

args = TrainingArguments(
    f"{model_checkpoint}-finetuned-pos",
    evaluation_strategy = "steps",
    eval_steps = eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
def calculate_parameters(ann):
    """Calculate trainable parameters"""
    model_num_param=0
    for p in ann.parameters():
        model_num_param += p.numel()
    
    return model_num_param

In [ ]:
def evaluate_dataset(df):
    predictions, labels, _ = trainer.predict(df)
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    return results

## Model training

### Full model fine-tuning

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

In [ ]:
calculate_parameters(model)

In [ ]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
evaluate_dataset(tokenized_datasets["test"])

### Training own adapter layer

In [ ]:
from transformers import ParallelConfig


model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

adapter_config = ParallelConfig()

model.add_adapter("pos", config=adapter_config)
model.set_active_adapters("pos")
model.train_adapter("pos")

In [ ]:
calculate_parameters(model)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
evaluate_dataset(tokenized_datasets["test"])

## Results

### Full model fine-tune (distilbert-base-uncased)
```
'overall_precision': 0.8918596918984817,
'overall_recall': 0.8993853031573065,
'overall_f1': 0.8956066887398793,
'overall_accuracy': 0.9222774670257579
```

### Adapter fine-tune (distilbert-base-uncased) (3.5M (or 5%) of trainable parameters)
```
'overall_precision': 0.8929514806061448,
'overall_recall': 0.8973176865046102,
'overall_f1': 0.8951292592850673,
'overall_accuracy': 0.9227090670533803
```

### Full model fine-tune (xlm-roberta-large)
```
N/A, it is possible to train on a single 16GB GPU, but batch size is only 4 which is practically not usable
```

### Adapter fine-tune (xlm-roberta-large) (25M (or 4.5%) of trainable parameters)
```
'overall_precision': 0.9267465013882267,
'overall_recall': 0.9232768014305273,
'overall_f1': 0.9250083977158213,
'overall_accuracy': 0.9399161058635362
```